# Init for pyqcu.

In [ ]:
import cupy as cp
import functools
import cupyx.scipy.sparse as cpx_sparse
from pyqcu import eigen, bistabcg

# Give matvec.

In [ ]:
index = -1
n = 16**3*3


def generate_sparse_complex_psd_matrix(n, density=0.1):
    real_part = cpx_sparse.random(
        n, n, density=density, format="csr", dtype=cp.float32)
    imag_part = cpx_sparse.random(
        n, n, density=density, format="csr", dtype=cp.float32)
    A = real_part + 1j * imag_part
    A_hermitian = A + A.getH()
    A_psd = A_hermitian + n * cpx_sparse.identity(n, dtype=cp.complex64)
    return A_psd


A = generate_sparse_complex_psd_matrix(n)
print(A.shape)


def matvec(src):
    return A@src

# Give guage's eigenvalues and eigenvectors

In [ ]:
eigen_solver = eigen.solver(
    n=n, k=10,matvec=matvec,dtype=A.dtype)
eigenvalues, eigenvectors = eigen_solver.run()

In [ ]:
print(eigenvalues)

# Run matvec(eigenvector[.]) ?= eigenvalue[.]*eigenvector[.] for eigen test.

In [ ]:
for i, ev in enumerate(eigenvalues):
    print(f"λ_{i} = {ev:.2e}")
    # Verify eigenvector
    v = eigenvectors[i]
    w = cp.zeros_like(v)
    w = matvec(v)
    error = cp.linalg.norm(w - ev * v) / cp.linalg.norm(w)
    print(f"Relative error: {error:.2e}")
    j = i+1
    if j == len(eigenvalues):
        j = 0
    print(
        f"Diff between λ_{i} and λ_{j}: {cp.linalg.norm(eigenvectors[i] - eigenvectors[j])/cp.linalg.norm(eigenvectors[i]):.2e}")

# Sovle (A-a)x+b = b by BISTABCG

In [ ]:
a = eigenvalues[index]
print(a)
b = cp.ones(n, dtype=A.dtype)

In [ ]:
def _matvec(src, a,b):
    return matvec(src)-a+b

In [ ]:

bistabcg_solver = bistabcg.slover(
    b=b, matvec=functools.partial(_matvec, a=a, b=b), max_iter=10000, tol=1e-4)
x = bistabcg_solver.run()

In [ ]:
b.flatten()[:50]

In [ ]:
print(b.shape)
bistabcg_solver = bistabcg.slover(
    b=b, matvec=matvec, max_iter=10000, tol=1e-9)
_x = bistabcg_solver.run()

# Verify above

In [ ]:
x.flatten()[:50]

In [ ]:
eigenvectors[index].flatten()[:50]

In [ ]:
Ax = matvec(x)
Ax.flatten()[:50]

In [ ]:
ax = a*x
ax.flatten()[:50]

In [ ]:
print(cp.linalg.norm(Ax-ax)/cp.linalg.norm(ax))

# End

In [ ]:
# bistabcg_solver.end()
